In [1]:
# import os
# import re
# import torch
# import torch.nn.functional as F

# import re
# from tqdm import tqdm
# import torch
# import torch.nn.functional as F
# import pandas as pd
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from langchain_text_splitters import RecursiveCharacterTextSplitter


# import re

# def split_sentence_with_indices(text):
#     pattern = r'''
#         (?:
#             \d+[.,]?\d*\s*[%$€]?
#         )                                     # Numbers with optional decimal/currency
#         |
#         [A-Za-zÀ-ÖØ-öø-ÿ0-9]+(?:-[A-Za-z0-9]+)*  # Words with optional hyphens (e.g., anti-TNF)
#         |
#         [()\[\]{}]                             # Parentheses and brackets
#         |
#         [^\w\s]                                # Other single punctuation marks
#     '''
#     return list(re.finditer(pattern, text, flags=re.VERBOSE))



# def write_annotations_to_file(data, file_path):
#     """
#     Writes annotation data to a TSV file.

#     Parameters:
#         data (list of dict): Each dict should have keys:
#             'filename', 'ann_id', 'label', 'start_span', 'end_span', 'text'
#         file_path (str): Path to the output file
#     """
#     header = ['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']

#     with open(file_path, 'w', encoding='utf-8') as f:
#         # Write the header
#         f.write('\t'.join(header) + '\n')
#         # Write each row
#         for entry in data:
#             row = [str(entry[key]) for key in header]
#             f.write('\t'.join(row) + '\n')


# def load_tsv_to_dataframe(file_path: str) -> pd.DataFrame:
#     """
#     Loads a TSV file with specific columns into a pandas DataFrame.

#     Expected columns:
#         filename, label, start_span, end_span, text, note

#     Args:
#         file_path (str): Path to the TSV file.

#     Returns:
#         pd.DataFrame: DataFrame containing the TSV data.
#     """
#     df = pd.read_csv(
#         file_path,
#         sep='\t',
#         dtype={
#             "filename": str,
#             "label": str,
#             "start_span": int,
#             "end_span": int,
#             "text": str,
#             "note": str
#         },
#         keep_default_na=False  # Prevents empty strings being converted to NaN
#     )
#     return df


# class PredictionNER:
#     def __init__(self, model_checkpoint, revision) -> None:
#         MAX_LENGTH = 500
#         OVERLAPPING_LEN = 12 

#         self.tokenizer = AutoTokenizer.from_pretrained(
#             model_checkpoint, revision=revision, is_split_into_words=True
#         )
#         self.model = AutoModelForTokenClassification.from_pretrained(
#             model_checkpoint, revision=revision
#         )
#         self.text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#             encoding_name='o200k_base',
#             separators=["\n\n\n", "\n\n", "\n", ".", ",", " ", ""],
#             keep_separator=False,
#             chunk_size=MAX_LENGTH,
#             chunk_overlap=OVERLAPPING_LEN,
#         )

#         ner_labels = list(self.model.config.id2label.values())
#         self.base_entity_types = sorted(
#             set(label[2:] for label in ner_labels if label != "O")
#         )


#     def split_text_with_indices(self, text):
#         offset = 0
#         for doc in self.text_splitter.split_text(text):
#             # Search for doc within the remaining text
#             start_idx = text.find(doc, offset)
#             if start_idx == -1:
#                 continue  # should not happen, but skip just in case
#             end_idx = start_idx + len(doc)
#             offset = end_idx  # move search window forward
#             yield doc, start_idx, end_idx

#     def predict_text(self, text: str, confidence_threshold: float = 0.7):
#         # 1. Split text into words and punctuation using regex
#         text_matches = split_sentence_with_indices(text)  # list(re.finditer(r'([0-9A-Za-zÀ-ÖØ-öø-ÿ]+|[^0-9A-Za-zÀ-ÖØ-öø-ÿ])', text))

#         # 2. Strip and filter out empty or whitespace-only tokens
#         text_words = [m.group().strip() for m in text_matches if m.group().strip()]

#         if not text_words:
#             return []  # return early if nothing valid

#         # 3. Tokenize with word alignment
#         inputs = self.tokenizer(
#             text_words, 
#             return_tensors="pt", 
#             is_split_into_words=True,
#             truncation=False
#         )
#         word_ids = inputs.word_ids()

#         # 4. Predict
#         with torch.no_grad():
#             logits = self.model(**inputs).logits
#             probs = F.softmax(logits, dim=-1)

#         predictions = torch.argmax(logits, dim=2)[0]

#         # 5. Map predictions back to original stripped words
#         results = []
#         seen = set()
#         non_empty_matches = [m for m in text_matches if m.group().strip()]

#         for i, word_idx in enumerate(word_ids):
#             if word_idx is None or word_idx in seen:
#                 continue
#             seen.add(word_idx)

#             word = text_words[word_idx]
#             tag_id = predictions[i].item()
#             tag = self.model.config.id2label[tag_id]
#             score = probs[0, i, tag_id].item()
#             start = non_empty_matches[word_idx].start()
#             end = non_empty_matches[word_idx].end()

#             # Apply the confidence threshold filter
#             if score < confidence_threshold:
#                 tag = "O"  # Assign "O" tag if confidence is below threshold
#                 score = 0.0  # Set score to 0 for "O" tag

#             results.append({
#                 'word': word,
#                 'tag': tag,
#                 'start': start,
#                 'end': end,
#                 'score': score
#             })

#         return results

#     def aggregate_entities(self, tagged_tokens, original_text, confidence_threshold=0.3):
#         # Step 1: Preprocess tags based on the two rules
#         corrected_tokens = tagged_tokens.copy()

#         # Rule 1: Fix "O" between "B-" and "I-" of the same type
#         for i in range(1, len(tagged_tokens) - 1):
#             prev_tag = tagged_tokens[i - 1]["tag"]
#             curr_tag = tagged_tokens[i]["tag"]
#             next_tag = tagged_tokens[i + 1]["tag"]

#             if (
#                 curr_tag == "O" and
#                 prev_tag.startswith("B-") and
#                 next_tag.startswith("I-")
#             ):
#                 prev_type = prev_tag[2:]
#                 next_type = next_tag[2:]
#                 if prev_type == next_type:
#                     corrected_tokens[i]["tag"] = "I-" + prev_type

#         # Rule 2: Convert isolated or starting I- to B-
#         last_tag_type = None
#         for i in range(len(corrected_tokens)):
#             tag = corrected_tokens[i]["tag"]
#             if tag.startswith("I-"):
#                 tag_type = tag[2:]
#                 if last_tag_type != tag_type:
#                     corrected_tokens[i]["tag"] = "B-" + tag_type
#                     last_tag_type = tag_type
#                 else:
#                     last_tag_type = tag_type
#             elif tag.startswith("B-"):
#                 last_tag_type = tag[2:]
#             else:
#                 last_tag_type = None

#         # Step 2: Apply original aggregation logic
#         entities = []
#         current_entity = None

#         for item in corrected_tokens:
#             tag = item["tag"]
#             start = item["start"]
#             end = item["end"]
#             score = item["score"]

#             if tag.startswith("B-"):
#                 if current_entity:
#                     if all(s >= confidence_threshold for s in current_entity["scores"]):
#                         current_entity["text"] = original_text[current_entity["start"]:current_entity["end"]]
#                         current_entity["score"] = sum(current_entity["scores"]) / len(current_entity["scores"])
#                         del current_entity["scores"]
#                         entities.append(current_entity)
#                     current_entity = None
#                 tag_type = tag[2:]
#                 current_entity = {
#                     "start": start,
#                     "end": end,
#                     "tag": tag_type,
#                     "scores": [score]
#                 }

#             elif tag.startswith("I-"):
#                 tag_type = tag[2:]
#                 if current_entity and current_entity["tag"] == tag_type:
#                     current_entity["end"] = end
#                     current_entity["scores"].append(score)
#                 else:
#                     current_entity = {
#                         "start": start,
#                         "end": end,
#                         "tag": tag_type,
#                         "scores": [score]
#                     }

#             else:  # "O"
#                 if current_entity:
#                     if all(s >= confidence_threshold for s in current_entity["scores"]):
#                         current_entity["text"] = original_text[current_entity["start"]:current_entity["end"]]
#                         current_entity["score"] = sum(current_entity["scores"]) / len(current_entity["scores"])
#                         del current_entity["scores"]
#                         entities.append(current_entity)
#                     current_entity = None

#         if current_entity:
#             if all(s >= confidence_threshold for s in current_entity["scores"]):
#                 current_entity["text"] = original_text[current_entity["start"]:current_entity["end"]]
#                 current_entity["score"] = sum(current_entity["scores"]) / len(current_entity["scores"])
#                 del current_entity["scores"]
#                 entities.append(current_entity)

#         return entities


#     def do_prediction(self, text, confidence_threshold=0.6):
#         final_prediction = []
#         for sub_text, sub_text_start, sub_text_end in self.split_text_with_indices(text):
#             tokens = self.predict_text(text=sub_text, confidence_threshold=confidence_threshold)
#             predictions = self.aggregate_entities(tokens, sub_text, confidence_threshold=confidence_threshold)


#             for pred in predictions:
#                 pred["start"] += sub_text_start
#                 pred["end"] += sub_text_start
#                 final_prediction.append(pred)

#         final_prediction_dict = {
#             lab: [p for p in final_prediction if p["tag"] == lab]
#             for lab in self.base_entity_types
#         }
#         merged_predictions = []
#         for label in self.base_entity_types:
#             merged_predictions.extend(final_prediction_dict[label])
#         return merged_predictions




In [2]:
model_checkpoint = "DT4H-IE/CardioBERTa.es_ES_MED"
revision = "2025-05-06_09-40-29-ed3a39e1" 

root = "../dataset/Spanish"
lang = "es"
cat = "med"

ner = PredictionNER(model_checkpoint=model_checkpoint, revision=revision)


In [4]:

# conver the predictions to ann format
tsv_file_path_test = os.path.join(root,  f"test_cardioccc_{lang}_{cat}.tsv")
test_files_root =  os.path.join(root, "txt")

test_df = load_tsv_to_dataframe(tsv_file_path_test)
prd_ann = []

for fn in tqdm(test_df['filename'].unique()):

    with open(os.path.join(test_files_root, fn+".txt"), 'r', encoding='utf-8') as f:
        document_text = f.read()
        prds = ner.do_prediction(document_text, confidence_threshold=0.35)
        for prd in prds:
            prd_ann.append({
                "filename": fn,
                "label": prd["tag"],
                "ann_id": "NA",
                "start_span": prd["start"],
                "end_span": prd["end"],
                "text": prd["text"],
            })

# [x['text'] for x in prd_ann]
write_annotations_to_file(prd_ann, os.path.join(root, f"pre_{model_checkpoint.split('/')[1]}_{revision}.tsv"))


  0%|          | 0/219 [00:00<?, ?it/s]


['atorvastatina',
 'estatina',
 'atenolol',
 'simvastatina',
 'atorvastatina',
 'ácido acetilsalicílico',
 'bisoprolol',
 'ramipril',
 'subtilisina/',
 'evolocumab',
 'atorvastatina']

In [8]:
text = """EVOLUCIÓN CLÍNICA
Paciente con antecedentes familiares de enfermedad coronaria precoz y sospecha de hipercolesterolemia familiar, asintomático desde el punto de vista cardiológico hasta el episodio actual, altamente sugestivo de angina de esfuerzo. Años antes había recibido tratamiento con estatinas, inicialmente simvastatina 20 pero se sustituyó por ineficacia (cLDL > 180 mg/d) por atorvastatina 10 que tuvo que suspenderse por daño hepático, reversible al interrumpir el tratamiento. Esto ocurrió en el año 2009 (en ese momento había bajado a LDL 129 mg/dl). Desde entonces, sin tratamiento hipolipemiante, ha mantenido niveles de cLDL entre 180 y 250 mg/dl. En la actualidad, consulta por un único dolor torácico en relación a un esfuerzo mayor de lo habitual; se descarta evento agudo y se realiza ergometría que es negativa 12 MET, pero debido a la probabilidad de enfermedad coronaria (síntomas típicos, historia familiar e hipercolesterolemia mantenida durante muchos años) se solicita angio-TC coronario, en el que se aprecian varias estenosis no significativas en el árbol coronario. A la vista de estos resultados, se inicia tratamiento antiagregante (ácido acetilsalicílico 100 mg), betabloqueante (bisoprolol 2,5 mg ) y ramipril (2,5 mg), además de tratamiento hipolipemiante en este caso con un inhibidor de la proproteína convertasa subtilisina / kexina tipo 9 (iPCSK9) (evolocumab 140 mg/2 semanas) por el antecedente de probable hipercolesterolemia familiar, intolerancia a la dosis inicial de atorvastatina e ineficacia con otra estatina menos potente y la presencia de enfermedad coronaria. Actualmente se encuentra pendiente de resultado de estudio genético.
"""
ner.do_prediction(text,0.35)

[{'start': 315,
  'end': 327,
  'tag': 'MEDICATION',
  'text': 'simvastatina',
  'score': 0.9982998967170715},
 {'start': 386,
  'end': 399,
  'tag': 'MEDICATION',
  'text': 'atorvastatina',
  'score': 0.9982932209968567},
 {'start': 1165,
  'end': 1187,
  'tag': 'MEDICATION',
  'text': 'ácido acetilsalicílico',
  'score': 0.9945134520530701},
 {'start': 1213,
  'end': 1223,
  'tag': 'MEDICATION',
  'text': 'bisoprolol',
  'score': 0.9983952641487122},
 {'start': 1235,
  'end': 1243,
  'tag': 'MEDICATION',
  'text': 'ramipril',
  'score': 0.9983286261558533},
 {'start': 1311,
  'end': 1320,
  'tag': 'MEDICATION',
  'text': 'inhibidor',
  'score': 0.6627407670021057},
 {'start': 1350,
  'end': 1363,
  'tag': 'MEDICATION',
  'text': 'subtilisina /',
  'score': 0.6465933620929718},
 {'start': 1388,
  'end': 1398,
  'tag': 'MEDICATION',
  'text': 'evolocumab',
  'score': 0.9982590079307556},
 {'start': 1513,
  'end': 1526,
  'tag': 'MEDICATION',
  'text': 'atorvastatina',
  'score': 0.9983

In [5]:
# To load pre-trained model training configs:

arg_path  = r"./trained_models/model_bert-base-spanish-wwm-cased_es_med_30042025_06bc94a6/training_args.bin"

import torch

# Load the full object (not just weights)
training_args = torch.load(arg_path, weights_only=False)

print(training_args)

In [5]:
import os
import re
import torch
import torch.nn.functional as F

from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from langchain_text_splitters import RecursiveCharacterTextSplitter
import argparse
from dotenv import find_dotenv, load_dotenv

print(load_dotenv(find_dotenv(".env")))

def split_sentence_with_indices(text):
    pattern = r'''
        (?:
            \d+[.,]?\d*\s*[%$€]?
        )                                     # Numbers with optional decimal/currency
        |
        [A-Za-zÀ-ÖØ-öø-ÿ0-9]+(?:-[A-Za-z0-9]+)*  # Words with optional hyphens (e.g., anti-TNF)
        |
        [()\[\]{}]                             # Parentheses and brackets
        |
        [^\w\s]                                # Other single punctuation marks
    '''
    return list(re.finditer(pattern, text, flags=re.VERBOSE))



def write_annotations_to_file(data, file_path):
    """
    Writes annotation data to a TSV file.

    Parameters:
        data (list of dict): Each dict should have keys:
            'filename', 'ann_id', 'label', 'start_span', 'end_span', 'text'
        file_path (str): Path to the output file
    """
    header = ['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']

    with open(file_path, 'w', encoding='utf-8') as f:
        # Write the header
        f.write('\t'.join(header) + '\n')
        # Write each row
        for entry in data:
            row = [str(entry[key]) for key in header]
            f.write('\t'.join(row) + '\n')


def load_tsv_to_dataframe(file_path: str) -> pd.DataFrame:
    """
    Loads a TSV file with specific columns into a pandas DataFrame.

    Expected columns:
        filename, label, start_span, end_span, text, note

    Args:
        file_path (str): Path to the TSV file.

    Returns:
        pd.DataFrame: DataFrame containing the TSV data.
    """
    df = pd.read_csv(
        file_path,
        sep='\t',
        dtype={
            "filename": str,
            "label": str,
            "start_span": int,
            "end_span": int,
            "text": str,
            "note": str
        },
        keep_default_na=False  # Prevents empty strings being converted to NaN
    )
    return df


class PredictionNER:
    def __init__(self, model_checkpoint, revision) -> None:
        MAX_LENGTH = 500
        OVERLAPPING_LEN = 12 

        self.tokenizer = AutoTokenizer.from_pretrained(
            model_checkpoint, revision=revision, is_split_into_words=True
        )
        self.model = AutoModelForTokenClassification.from_pretrained(
            model_checkpoint, revision=revision
        )
        self.text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            encoding_name='o200k_base',
            separators=["\n\n\n", "\n\n", "\n", ".", ",", " ", ""],
            keep_separator=False,
            chunk_size=MAX_LENGTH,
            chunk_overlap=OVERLAPPING_LEN,
        )

        ner_labels = list(self.model.config.id2label.values())
        self.base_entity_types = sorted(
            set(label[2:] for label in ner_labels if label != "O")
        )


    def split_text_with_indices(self, text):
        offset = 0
        for doc in self.text_splitter.split_text(text):
            # Search for doc within the remaining text
            start_idx = text.find(doc, offset)
            if start_idx == -1:
                continue  # should not happen, but skip just in case
            end_idx = start_idx + len(doc)
            offset = end_idx  # move search window forward
            yield doc, start_idx, end_idx

    def predict_text(self, text: str, confidence_threshold: float = 0.7):
        # 1. Split text into words and punctuation using regex
        text_matches = split_sentence_with_indices(text)  # list(re.finditer(r'([0-9A-Za-zÀ-ÖØ-öø-ÿ]+|[^0-9A-Za-zÀ-ÖØ-öø-ÿ])', text))

        # 2. Strip and filter out empty or whitespace-only tokens
        text_words = [m.group().strip() for m in text_matches if m.group().strip()]

        if not text_words:
            return []  # return early if nothing valid

        # 3. Tokenize with word alignment
        inputs = self.tokenizer(
            text_words, 
            return_tensors="pt", 
            is_split_into_words=True,
            truncation=False
        )
        word_ids = inputs.word_ids()

        # 4. Predict
        with torch.no_grad():
            logits = self.model(**inputs).logits
            probs = F.softmax(logits, dim=-1)

        predictions = torch.argmax(logits, dim=2)[0]

        # 5. Map predictions back to original stripped words
        results = []
        seen = set()
        non_empty_matches = [m for m in text_matches if m.group().strip()]

        for i, word_idx in enumerate(word_ids):
            if word_idx is None or word_idx in seen:
                continue
            seen.add(word_idx)

            word = text_words[word_idx]
            tag_id = predictions[i].item()
            tag = self.model.config.id2label[tag_id]
            score = probs[0, i, tag_id].item()
            start = non_empty_matches[word_idx].start()
            end = non_empty_matches[word_idx].end()

            # Apply the confidence threshold filter
            if score < confidence_threshold:
                tag = "O"  # Assign "O" tag if confidence is below threshold
                score = 0.0  # Set score to 0 for "O" tag

            results.append({
                'word': word,
                'tag': tag,
                'start': start,
                'end': end,
                'score': score
            })

        return results

    def aggregate_entities(self, tagged_tokens, original_text, confidence_threshold=0.3):
        # Step 1: Preprocess tags based on the two rules
        corrected_tokens = tagged_tokens.copy()

        # Rule 1: Fix "O" between "B-" and "I-" of the same type
        for i in range(1, len(tagged_tokens) - 1):
            prev_tag = tagged_tokens[i - 1]["tag"]
            curr_tag = tagged_tokens[i]["tag"]
            next_tag = tagged_tokens[i + 1]["tag"]

            if (
                curr_tag == "O" and
                prev_tag.startswith("B-") and
                next_tag.startswith("I-")
            ):
                prev_type = prev_tag[2:]
                next_type = next_tag[2:]
                if prev_type == next_type:
                    corrected_tokens[i]["tag"] = "I-" + prev_type

        # Rule 2: Convert isolated or starting I- to B-
        last_tag_type = None
        for i in range(len(corrected_tokens)):
            tag = corrected_tokens[i]["tag"]
            if tag.startswith("I-"):
                tag_type = tag[2:]
                if last_tag_type != tag_type:
                    corrected_tokens[i]["tag"] = "B-" + tag_type
                    last_tag_type = tag_type
                else:
                    last_tag_type = tag_type
            elif tag.startswith("B-"):
                last_tag_type = tag[2:]
            else:
                last_tag_type = None

        # Step 2: Apply original aggregation logic
        entities = []
        current_entity = None

        for item in corrected_tokens:
            tag = item["tag"]
            start = item["start"]
            end = item["end"]
            score = item["score"]

            if tag.startswith("B-"):
                if current_entity:
                    if all(s >= confidence_threshold for s in current_entity["scores"]):
                        current_entity["text"] = original_text[current_entity["start"]:current_entity["end"]]
                        current_entity["score"] = sum(current_entity["scores"]) / len(current_entity["scores"])
                        del current_entity["scores"]
                        entities.append(current_entity)
                    current_entity = None
                tag_type = tag[2:]
                current_entity = {
                    "start": start,
                    "end": end,
                    "tag": tag_type,
                    "scores": [score]
                }

            elif tag.startswith("I-"):
                tag_type = tag[2:]
                if current_entity and current_entity["tag"] == tag_type:
                    current_entity["end"] = end
                    current_entity["scores"].append(score)
                else:
                    current_entity = {
                        "start": start,
                        "end": end,
                        "tag": tag_type,
                        "scores": [score]
                    }

            else:  # "O"
                if current_entity:
                    if all(s >= confidence_threshold for s in current_entity["scores"]):
                        current_entity["text"] = original_text[current_entity["start"]:current_entity["end"]]
                        current_entity["score"] = sum(current_entity["scores"]) / len(current_entity["scores"])
                        del current_entity["scores"]
                        entities.append(current_entity)
                    current_entity = None

        if current_entity:
            if all(s >= confidence_threshold for s in current_entity["scores"]):
                current_entity["text"] = original_text[current_entity["start"]:current_entity["end"]]
                current_entity["score"] = sum(current_entity["scores"]) / len(current_entity["scores"])
                del current_entity["scores"]
                entities.append(current_entity)

        return entities


    def do_prediction(self, text, confidence_threshold=0.6):
        final_prediction = []
        for sub_text, sub_text_start, sub_text_end in self.split_text_with_indices(text):
            tokens = self.predict_text(text=sub_text, confidence_threshold=confidence_threshold)
            predictions = self.aggregate_entities(tokens, sub_text, confidence_threshold=confidence_threshold)


            for pred in predictions:
                pred["start"] += sub_text_start
                pred["end"] += sub_text_start
                final_prediction.append(pred)

        final_prediction_dict = {
            lab: [p for p in final_prediction if p["tag"] == lab]
            for lab in self.base_entity_types
        }
        merged_predictions = []
        for label in self.base_entity_types:
            merged_predictions.extend(final_prediction_dict[label])
        return merged_predictions


def evaluate(model_checkpoint, revision, root_path, lang, cat):

    ner = PredictionNER(model_checkpoint=model_checkpoint, revision=revision)

    # conver the predictions to ann format
    tsv_file_path_test = os.path.join(root_path,  f"test_cardioccc_{lang}_{cat}.tsv")
    test_files_root =  os.path.join(root_path, "txt")

    test_df = load_tsv_to_dataframe(tsv_file_path_test)
    prd_ann = []

    for fn in tqdm(test_df['filename'].unique()):

        with open(os.path.join(test_files_root, fn+".txt"), 'r', encoding='utf-8') as f:
            document_text = f.read()
            prds = ner.do_prediction(document_text, confidence_threshold=0.35)
            for prd in prds:
                prd_ann.append({
                    "filename": fn,
                    "label": prd["tag"],
                    "ann_id": "NA",
                    "start_span": prd["start"],
                    "end_span": prd["end"],
                    "text": prd["text"],
                })

    output_tsv_path = os.path.join(root_path, f"pre_{model_checkpoint.split('/')[1]}_{revision}.tsv")
    write_annotations_to_file(prd_ann, output_tsv_path)
    print(f"output_tsv_path {output_tsv_path}")

    

True


In [6]:
model_checkpoint = "DT4H-IE/CardioBERTa.en_EN_MED"
revision = "2025-05-07_19-08-13-d46774ee"
root = "dataset/English" 
cat = "med"
lang = "en"

evaluate(model_checkpoint, revision, root, lang, cat)


ValueError: Unrecognized model in DT4H-IE/CardioBERTa.en_EN_MED. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v3, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, encodec, en